In [1]:
import numpy as np 
import pickle as pkl 
import os

# Create 32x32 data with labels 

In [2]:
destination_directory = '' # the path to where the downloads are saved. http://bias.csr.unibo.it/maltoni/download/core50/core50_imgs.npz and https://vlomonaco.github.io/core50/data/paths.pkl
pkl_file = open(os.path.join(destination_directory, 'paths.pkl') , 'rb') 
paths = pkl.load(pkl_file) 

In [3]:
imgs = np.load(os.path.join(destination_directory, 'core50_imgs.npz'))['x'] 

In [4]:

import re

pattern = r'C_(\d+)_(\d+)_(\d+)\.png'

def parse(filename):
    match = re.match(pattern, filename)

    if match:
        session_num = match.group(1)
        obj_num = match.group(2)
        frame_seq_id = match.group(3)
        return session_num, obj_num, frame_seq_id
    else:
        print("No match found.")
        # AssertionError()

In [5]:
filter = r's(\d+)/o(\d+)/'
def filter_dir(filename):
    filename = re.sub(filter, '', filename)
    return filename

In [6]:
def object2class(obj_id):
    return (int(obj_id) - 1) // 5

In [7]:
session_ids, obj_ids, frame_ids, class_ids = [], [], [], []
for path in paths:
    session_id, obj_id, frame_id = parse(filter_dir(path))
    session_ids.append(int(session_id) - 1)
    obj_ids.append(int(obj_id) - 1)
    frame_ids.append(int(frame_id))
    class_ids.append(object2class(obj_id))

## Resize to 32 x 32

In [8]:
from PIL import Image
import torchvision.transforms as T

In [43]:
img = Image.fromarray(imgs[0])

In [9]:
data_32x32 = []
for img in imgs:
    img = Image.fromarray(img)
    resized_imgs = T.Resize(size=32)(img)
    resized_imgs = np.asarray(resized_imgs)
    data_32x32.append(resized_imgs)

## Save to pkl

In [10]:
core = {
    'data': data_32x32,
    'session': session_ids,
    'object': obj_ids,
    'frame': frame_ids,
    'category': class_ids
}


In [11]:
with open('core_data.pkl', 'wb') as f:
    pkl.dump(core,f)